Imports.

In [10]:
import sys
import warnings
from pathlib import Path
import numpy as np
import pandas as pd

In [11]:
D_ECHEM_LABELS = dict(time="time/s", 
                      voltage="Ewe/V",
                      current="<I>/mA",
                      x="x",
                      capacity="Capacity/mA.h",
                      cycle_number="cycle number",
                      )

Ignore UserWarnings in case `sys.exit()` is encountered.

In [12]:
warnings.filterwarnings("ignore", category=UserWarning)

Checking whether a folder called `data` exists.

In [13]:
data_path = Path.cwd() / "data"
if not data_path.exists():
    data_path.mkdir()
    s = f"\n{80*'-'}\nA folder called '{data_path.name}' has been created."
    s += f"\nPlease put your data files there and rerun the code.\n{80*'-'}"
    sys.exit(s)

Checking that data files are present in the `data` folder.

In [14]:
data_files = list(data_path.glob("*.*"))
if len(data_files) == 0:
    s = f"\n{80*'-'}\nNo files were found in the '{data_path.name}' folder.\n"
    s += f"Please put your data files there and rerun the code.\n{80*'-'}"
    sys.exit(s)

In [15]:
def get_header(file):
    with file.open(mode="r") as f:
        lines = f.readlines()
    if "EC-Lab" in lines[0]:
        header_idx = int(lines[1].split()[-1]) - 1
    else:
        header_idx = 0
    
    return lines[header_idx].split(), header_idx

In [16]:
def write(df, d, filename, output_path):
    time = df[d["time"]].to_numpy()
    time = (time - time[0]) / 60
    current = df[d["current"]].to_numpy() * 10**-3
    voltage, x = df[d["voltage"]].to_numpy(), df["x"].to_numpy()
    header = "t [h]\tV [V]\ti [A]\tx"
    np.savetxt(output_path / filename,
               np.column_stack((time, voltage, current, x)),
               delimiter="\t",
               header=header,
               fmt="%.8f",
               encoding="utf8",
               )
    
    return None

In [17]:
txt_path = Path.cwd() / "txt"
if not txt_path.exists():
    txt_path.mkdir()

In [18]:
print(f"{80*'-'}\nQuantities and labels to look for in data file(s):")
for k, v in D_ECHEM_LABELS.items():
    if len(k) < 8:
        print(f"\t{k}\t\t{v}")
    else:
        print(f"\t{k}\t{v}")
print(f"{80*'-'}\nIf other quantities and/or keys a desired, please edit in the "
      f"top of this .ipynb\nfile.\n{80*'-'}\nExtracting data from files...")
for file in data_files:
    print(f"\t{file.name}")
    header, header_idx = get_header(file)
    df = pd.read_csv(file, 
                     skiprows=header_idx, 
                     delimiter="\t", 
                     decimal=",",
                     )
    for k in df.keys():
        if "Unnamed" in k:
            df.pop(k)
    write(df, D_ECHEM_LABELS, file.name, txt_path)
print(f"Done extracting data from files. Please see the '{txt_path.name} "
      f"folder.\n{80*'-'}")

--------------------------------------------------------------------------------
Quantities and labels to look for in data file(s):
	time		time/s
	voltage		Ewe/V
	current		<I>/mA
	x		x
	capacity	Capacity/mA.h
	cycle_number	cycle number
--------------------------------------------------------------------------------
If other quantities and/or keys a desired, please edit in the top of this .ipynb
file.
--------------------------------------------------------------------------------
Extracting data from files...
	02_BIOLOGIC_Na07Fe033Mn067O2-Na_6dot29.txt


	03_BIOLOGIC_Na07Fe033Mn067O2-Na_6dot29_header.txt
Done extracting data from files. Please see the 'txt folder.
--------------------------------------------------------------------------------
